# Draw the concept graph
Open rviz and use the rviz config `bagplay_concept.rviz`. You can launch ros core and rviz from the `robotparam_and_rviz.launch`

The code loads a recorded rosbag, and pass the topics into registered callback functions. You can modify the following callbacks to alter the style of the visulization. What's more, you can choose a good period for the replay.

In [1]:
import rospy # for `Duration`
import rosbag
from semantic_front_end_filter.common import RosbagPlayer

In [2]:
# Import the modules for the model
from semantic_front_end_filter.adabins.pointcloudUtils import RaycastCamera
from semantic_front_end_filter.adabins.models import UnetAdaptiveBins
from semantic_front_end_filter.adabins.model_io import load_checkpoint, load_param_from_path

import torch 
import numpy as np
from semantic_front_end_filter.Labelling.messages.imageMessage import Camera, getImageId, rgb_msg_to_image
from semantic_front_end_filter.Labelling.messages.pointcloudMessage import rospcmsg_to_pcarray, ros_pc_msg2
from semantic_front_end_filter.Labelling.messages.messageToVectors import msg_to_body_ang_vel, msg_to_body_lin_vel, msg_to_rotmat, msg_to_command, \
    msg_to_pose, msg_to_joint_positions, msg_to_joint_velocities, msg_to_joint_torques, msg_to_grav_vec

In [3]:
rosbagpath = "/Data/Italy_0820/Reconstruct_2022-07-18-20-34-01_0.bag"
player = RosbagPlayer(rosbagpath)

/home/chenyu/projects/kpconv/semantic_front_end_filter/semantic_front_end_filter/common/rosbagPlayer.py:22: UserWarning: translation should be of type Vector3
  self.tf_buffer.set_transform_static(transform, 'rosbag')
/home/chenyu/projects/kpconv/semantic_front_end_filter/semantic_front_end_filter/common/rosbagPlayer.py:22: UserWarning: rotation should be of type Quaternion
  self.tf_buffer.set_transform_static(transform, 'rosbag')
/home/chenyu/projects/kpconv/semantic_front_end_filter/semantic_front_end_filter/common/rosbagPlayer.py:27: UserWarning: translation should be of type Vector3
  self.tf_buffer.set_transform(transform, 'rosbag')
/home/chenyu/projects/kpconv/semantic_front_end_filter/semantic_front_end_filter/common/rosbagPlayer.py:27: UserWarning: rotation should be of type Quaternion
  self.tf_buffer.set_transform(transform, 'rosbag')


In [4]:
rospy.init_node('draw_elev_map', anonymous=False)
player.add_publisher_of_topic("/alphasense_driver_ros/imu")
player.add_publisher_of_topic("/tf", queue_size=1000)
player.add_publisher_of_topic("/tf_static")
player.add_publisher_of_topic("/clock")
player.add_publisher_of_topic("/twist_mux/twist")


In [9]:
# player._callbacks = {k:v for k,v in player._callbacks.items() if k in ["/tf", "/tf_static","/clock"]}
# player.play(end_time = player.bag.get_start_time()+10)
player.play(start_time= player.bag.get_start_time()+90,
            end_time = player.bag.get_start_time()+95)


In [6]:
torch.cuda.empty_cache()# this is useless

In [7]:
modelname = "2022-08-29-23-51-44"
model_path = f"../checkpoints/{modelname}/UnetAdaptiveBins_best.pt"
image_topic = "/alphasense_driver_ros/cam4/debayered"
pc_topic = "/bpearl_rear/point_cloud"
TF_BASE = "base"
TF_MAP = "map"

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.cuda.empty_cache()
raycastCamera = RaycastCamera(device=device) # WARN: This raycastcamera is hard coded with `tf_base_to_sensor`, however, it seems to be constant


## Initialize model
model_cfg = load_param_from_path(model_path)
model_cfg["input_channel"] = 4
model = UnetAdaptiveBins.build(**model_cfg)

model = load_checkpoint(model_path, model)[0]
model.to(device)

Loading base model ()...Done.
Removing last two layers (global_pool & classifier).
Building Encoder-Decoder model..Done.


UnetAdaptiveBins(
  (encoder): Encoder(
    (original_model): GenEfficientNet(
      (conv_stem): Conv2d(4, 48, kernel_size=(3, 3), stride=(2, 2), bias=False)
      (bn1): BatchNorm2d(48, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
      (act1): SiLU(inplace=True)
      (blocks): Sequential(
        (0): Sequential(
          (0): DepthwiseSeparableConv(
            (conv_dw): Conv2d(48, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=48, bias=False)
            (bn1): BatchNorm2d(48, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
            (act1): SiLU(inplace=True)
            (se): SqueezeExcite(
              (conv_reduce): Conv2d(48, 12, kernel_size=(1, 1), stride=(1, 1))
              (act1): SiLU(inplace=True)
              (conv_expand): Conv2d(12, 48, kernel_size=(1, 1), stride=(1, 1))
            )
            (conv_pw): Conv2d(48, 24, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (bn2): BatchNorm2d(24, eps=0.0

In [16]:
from sensor_msgs import point_cloud2
from sensor_msgs.msg import PointCloud2, PointField
from std_msgs.msg import Header
from sensor_msgs.msg import Image
from visualization_msgs.msg import Marker
from geometry_msgs.msg import Point
import struct
from cv_bridge import CvBridge
image_cv_bridge = CvBridge()
def buildPoint(x, y, z, r, g, b, a=None):
    if(np.array([r, g, b]).max() < 1.01):
        r = int(r * 255.0)
        g = int(g * 255.0)
        b = int(b * 255.0)
        a = 255 if a is None else int(a * 255.0)
    else:
        r = int(r)
        g = int(g)
        b = int(b)
        a = 255 if a is None else int(a)
    rgb = struct.unpack('I', struct.pack('BBBB', b, g, r, a))[0]
    return [x, y, z, rgb]
rospy.init_node('draw_elev_map', anonymous=False)
pred_pc_pub1 = rospy.Publisher("pc/pred_pc/pub1", PointCloud2, queue_size=1)
raw_pc_pub1 = rospy.Publisher("pc/raw_pc/pub1", PointCloud2, queue_size=1)
pred_pc_pub2 = rospy.Publisher("pc/pred_pc/pub2", PointCloud2, queue_size=1)
raw_pc_pub2 = rospy.Publisher("pc/raw_pc/pub2", PointCloud2, queue_size=1)
image_pub = rospy.Publisher("cam_image", Image, queue_size=1)
marker_pub = rospy.Publisher("pred_raw_connection", Marker, queue_size=1)

## Configs of the line_list style
line_list_msg = Marker()
line_list_msg.header.frame_id = "map"
line_list_msg.ns = "raw2pred"
line_list_msg.action = 0
line_list_msg.type = 5
line_list_msg.color.a = 0.2
line_list_msg.color.r = 0.5
line_list_msg.color.b = 0.2
line_list_msg.color.g = 0.2
line_list_msg.scale.x = 0.005
line_list_msg.points = []


player._shared_var.update(dict(
    pcbuffer=[],
))

def pred_and_checkerr(image, pc, pose,v):
    """
    Make the prediction based on image, pointcloud and robot current pose
    arg pose: x,y,z,rx,ry,rz,rw
    """
    image = torch.Tensor(image).to(device)
    points = torch.Tensor(pc).to(device)
#     points_highlight = points.copy()
    highlight_mask = (points[:,2]>pose[2]-0.2)\
            & (torch.sum((points[:,:2]-torch.tensor(pose[:2]).to(device))**2, axis = 1)<2**2)

    points_highlight = points[highlight_mask] 
    
    # get pc image
    pc_img = torch.zeros_like(image[:1, ...],dtype = torch.float32).to(device)
    pc_img,visible,proj_point = raycastCamera.project_cloud_to_depth(
                    pose, points, pc_img, return_detail=True)
    pc_img_highlight = torch.zeros_like(image[:1, ...],dtype = torch.float32).to(device)
    pc_img_highlight = raycastCamera.project_cloud_to_depth(
                    pose, points_highlight, pc_img_highlight)
    # filter the points to the only visble ones
    points = points[visible] # TODO: Decide whether should be filter the raw points
    # get prediction
    model_in = torch.cat([image/255., pc_img.clone()],axis=0)
    model_in = model_in[None, ...]
    for i, (m, s) in enumerate(zip([0.387, 0.394, 0.404, 0.120], [0.322, 0.32, 0.30,  1.17])):
        model_in[0, i, ...] = (model_in[0, i, ...] - m)/s
    pred = model(model_in)[0][0]

    pred [(pc_img[0]==0)] = np.nan
    pred = pred.T

    # get elevation from prediction
    highlight_mask = abs(pc_img_highlight.T.reshape(-1))>1e-9
    
    pred_pts = raycastCamera.project_depth_to_cloud(pose, pred)
    pred_pts_highlight = pred_pts[highlight_mask]
    pred_pts = pred_pts[~torch.isnan(pred_pts[:,0])]
    # pred_pts = pred_pts[pred_pts[:,2]<pose[2]] # TODO: Decide whether this hieight mask is necessary
    pred_points = pred_pts.detach().cpu().numpy()
    pred_highlight_nanmask = torch.isnan(pred_pts_highlight[:,0])
    pred_points_highlight = pred_pts_highlight[~pred_highlight_nanmask].detach().cpu().numpy()
    
    
## Deprectaed way of getting raw points and colors
#     raw_points = points.cpu().numpy().astype(pred_points.dtype) # float_64
#     proj_point = proj_point.cpu().numpy()
#     colors_raw = np.moveaxis((image.cpu().numpy())[:3,proj_point[:,1], proj_point[:,0]], 0, 1)

    # Use reprojected raw_pts, otherwise the position of raw pc and pred pc are not aligned
    pc_img[abs(pc_img)<1e-9] = np.nan
    raw_pts = raycastCamera.project_depth_to_cloud(pose, pc_img.T)
    raw_pts_highlight = raw_pts[highlight_mask]
    raw_rgb = torch.vstack([(image[cnl].T).reshape(-1) for cnl in range(3)]).T
    raw_rgb = (raw_rgb-raw_rgb.min())/(raw_rgb.max() - raw_rgb.min())
    raw_rgb_highlight = raw_rgb[highlight_mask]
    raw_rgb = raw_rgb[~torch.isnan(raw_pts[:,0])]
    raw_pts = raw_pts[~torch.isnan(raw_pts[:,0])]
    raw_points = raw_pts.cpu().numpy()
    raw_colors = raw_rgb.cpu().numpy()
    
    raw_highlight_nanmask = torch.isnan(raw_pts_highlight[:,0])
    raw_points_highlight = raw_pts_highlight[~raw_highlight_nanmask].cpu().numpy()
    raw_colors_highlight = raw_rgb_highlight[~raw_highlight_nanmask].cpu().numpy()
    
    
    header = Header()
    header.frame_id = "map"
    fields = [
        PointField('x', 0, PointField.FLOAT32, 1),
        PointField('y', 4, PointField.FLOAT32, 1),
        PointField('z', 8, PointField.FLOAT32, 1),
        # PointField('rgb', 12, PointField.UINT32, 1),
        PointField('rgba', 12, PointField.UINT32, 1),
    ]
    
    color_pred_high = np.array([[255,170,0.]])/256
    color_pred_low = np.array([[80,60,0.]])/256
    colors_pred = np.clip(1.5*(pred_points[:, 2]-pred_points[:, 2].min())
        /(pred_points[:, 2].max()-pred_points[:, 2].min()),0.,1.)[:,None] \
        * (color_pred_high- color_pred_low) + color_pred_low
    cloud_pred = point_cloud2.create_cloud(header, fields,
                                [buildPoint(*p[:3], *c) for p, c in zip(pred_points, colors_pred)])
    cloud_pred_highlight = point_cloud2.create_cloud(header, fields,
                                [buildPoint(*p[:3], 0.,0.,0.) for p in pred_points_highlight])
    pred_pc_pub1.publish(cloud_pred)
    pred_pc_pub2.publish(cloud_pred_highlight)
    

    
    cloud_raw = point_cloud2.create_cloud(header, fields,
                                [buildPoint(*p[:3], *c) for p, c in zip(raw_points, raw_colors)])
    cloud_raw_highlight = point_cloud2.create_cloud(header, fields,
                                [buildPoint(*p[:3], *c) for p, c in zip(raw_points_highlight, raw_colors_highlight)])
    
    raw_pc_pub1.publish(cloud_raw)
    raw_pc_pub2.publish(cloud_raw_highlight)
    
    line_list_msg.points = [ p
        for pp, rp in zip(pred_points_highlight[::10], raw_points_highlight[::10])
        for p in [Point(*pp[:3]), Point(*rp[:3])]
    ]
    marker_pub.publish(line_list_msg)
    
    image_to_show = np.moveaxis(image.cpu().numpy(), 0, 2).astype(np.uint8)
    image_pub.publish(image_cv_bridge.cv2_to_imgmsg(image_to_show, "bgr8"))
#     print("image_to_show", image_to_show.shape, image_to_show.mean())

def image_cb(topic, msg, t, tf_buffer, v):
    if(not len(v["pcbuffer"])): return

    img = rgb_msg_to_image(msg, raycastCamera.camera.is_debayered, raycastCamera.camera.rb_swap, ("compressed" in topic))
    img = np.moveaxis(img, 2, 0)

    if not (tf_buffer.can_transform_core(TF_MAP, TF_BASE,  msg.header.stamp)[0]): return 
    tf = tf_buffer.lookup_transform_core(TF_MAP, TF_BASE, msg.header.stamp)
    pose = msg_to_pose(tf)  # pose in fixed ref frame (odom or map)

    pc = np.concatenate(v["pcbuffer"],axis = 0)
    pred_and_checkerr(img, pc, pose, v)

#     v["pcbuffer"] = v["pcbuffer"][-10:]
    v["pcbuffer"] = v["pcbuffer"][-2:]

player.register_callback(image_topic, image_cb)


def pointcloud_cb(topic, msg, t, tf_buffer, v):
    if not (tf_buffer.can_transform_core(TF_MAP, msg.header.frame_id,  msg.header.stamp)[0]): return
    tf = tf_buffer.lookup_transform_core(TF_MAP, msg.header.frame_id,  msg.header.stamp)
    pose = msg_to_pose(tf)
    pc_array = rospcmsg_to_pcarray(msg, pose)

    v["pcbuffer"].append(pc_array[:,:3])
player.register_callback(pc_topic, pointcloud_cb)


In [18]:
player.play(start_time= player.bag.get_start_time()+90,
            end_time = player.bag.get_start_time()+95)

# Draw the foot holds

In [20]:
import rospy 
import numpy as np
from semantic_front_end_filter.Labelling.GroundfromTrajs import GFT
from visualization_msgs.msg import Marker, MarkerArray
from geometry_msgs.msg import Pose, Quaternion
from std_msgs.msg import Header, Float32MultiArray, MultiArrayLayout, MultiArrayDimension
from geometry_msgs.msg import Point

foottrajpath = "/Data/Italy_0820/FeetTrajs.msgpack"

gft = GFT(FeetTrajsFile = foottrajpath, InitializeGP=False)
foot_holds = {k : np.array(gft.getContactPoints(v)[0]) for k,v in gft.FeetTrajs.items()} # A dict of the contact points of each foot
foot_holds_array = np.vstack(list(foot_holds.values()))
# rosbagpath = "/Data/Italy_0820/Reconstruct_2022-07-18-20-34-01_0.bag"
rospy.init_node('draw_elev_map', anonymous=False)

File is loaded successfully.
Extracting Contact points ......
Extracting Contact points ......
Extracting Contact points ......
Extracting Contact points ......


/home/chenyu/projects/kpconv/semantic_front_end_filter/semantic_front_end_filter/Labelling/GroundfromTrajs.py:175: RuntimeWarning: invalid value encountered in true_divide
  GroundArray = np.true_divide(GroundArray, CountContactArray)


Extracting Contact points ......
Extracting Contact points ......
Extracting Contact points ......
Extracting Contact points ......


In [21]:
marker_pub = rospy.Publisher("labeling/footholds", Marker, queue_size=1)

line_list_msg = Marker()
line_list_msg.header.frame_id = "map"
line_list_msg.ns = "footholds"
line_list_msg.action = 0
line_list_msg.type = Marker.SPHERE_LIST
line_list_msg.color.a = 1
line_list_msg.color.r = 0.1
line_list_msg.color.b = 0.5
line_list_msg.color.g = 0.2
line_list_msg.scale.x = 0.15
line_list_msg.points = []

line_list_msg.points = [Point(*p[:3]) for p in foot_holds_array]

marker_pub.publish(line_list_msg)

In [26]:
axis_pub = rospy.Publisher("labeling/convenientaxis", MarkerArray, queue_size=1)
marr = MarkerArray()
marr.markers = [
    Marker(
        header = Header(frame_id = "map"),
        ns = "axis",
        color = type(Marker().color)(
            0,0,0,1
        ),
        scale = type(Marker().scale)(
            8, 0.1, 0.1
        ),
        pose = Pose(
            position = Point(-16, 38, -8)
        ),
        id = i
        
    )
    for i in range(3)
]
marr.markers[0].type = Marker.ARROW
marr.markers[1].type = Marker.ARROW
marr.markers[2].type = Marker.ARROW


marr.markers[0].pose.orientation = Quaternion(0,0,0,1)
marr.markers[1].pose.orientation = Quaternion(0,0,1/np.sqrt(2), -1/np.sqrt(2))
marr.markers[2].pose.orientation = Quaternion(0,-1/np.sqrt(2), 0, 1/np.sqrt(2))
axis_pub.publish(marr)

### Draw the reconstructed surface with gridmap

In [3]:
from grid_map_msgs.msg import GridMap, GridMapInfo
import msgpack
import msgpack_numpy as m
m.patch()

groundmappath = "/Data/Italy_0820/GroundMap.msgpack"
with open(groundmappath, "rb") as data_file:
    data = data_file.read()
    ground_dict = msgpack.unpackb(data)
    print("load ground dict, y real range: ",ground_dict["yRealRange"], 
                            "x real range: ", ground_dict["xRealRange"])
    ground_dict = ground_dict




gp_surface_pub = rospy.Publisher("labeling/grid_map", GridMap, queue_size=1)
    
    


load ground dict, y real range:  [-3.295388865615678, 105.15809558634709] x real range:  [-108.38099452324194, 3.711307624349044]


In [10]:
ground_dict.keys()

dict_keys(['res', 'xNormal', 'yNormal', 'xRealRange', 'yRealRange', 'meanHeight', 'GroundArray', 'GPMap', 'Confidence'])

In [47]:
gpmap = np.array(ground_dict["GPMap"])
gpmap = gpmap[::-1, ::-1]
gpmap = gpmap.T

gpmapmsg = GridMap(
    info = GridMapInfo(
        header= Header(frame_id="map"),
        resolution = ground_dict["res"],
        length_x = ground_dict["res"] * gpmap.shape[1],
        length_y = ground_dict["res"] * gpmap.shape[0],
        pose = Pose(
#             position = Point(
#                 ground_dict["xRealRange"][1] - ground_dict["res"] * gpmap.shape[0]/2,
#                 ground_dict["yRealRange"][1] - ground_dict["res"] * gpmap.shape[1]/2,0),
            position = Point(
                (gpmap.shape[1]/2+ground_dict["xNormal"])*ground_dict["res"],
                (gpmap.shape[0]/2+ground_dict["yNormal"])*ground_dict["res"],0),
            orientation = Quaternion(x=0,y=0,z=0,w=1)
        )),
    layers = ["elevation"],
    basic_layers = ["elevation"],
    data = [
        Float32MultiArray(
            layout = MultiArrayLayout(
                dim = [
                    MultiArrayDimension(
                        label= ["column_index", "row_index"][i],
                        size = d,
                        stride = np.prod( gpmap.shape[i:])
                    )for i,d in enumerate(gpmap.shape)
                ]
            ),
            data = gpmap.reshape(-1).tolist()
            
        )
    ]
)
gp_surface_pub.publish(gpmapmsg)


#### Color conversion in gridmap

grid_map use a float number to represent the rgb color of the map. The conversion [code](http://docs.ros.org/en/jade/api/grid_map_core/html/GridMapMath_8cpp_source.html) defns the function

```c++
bool colorVectorToValue(const Eigen::Vector3i& colorVector, unsigned long& colorValue)
{
  colorValue = ((int)colorVector(0)) << 16 | ((int)colorVector(1)) << 8 | ((int)colorVector(2));
  return true;
}
```

The way to define color should be 
```python
color = np.array([10*2**16 + 200*2**8 + 123],dtype = np.uint32)
color.view(np.float32)
```

## Annotate position elevation map
Draw the scale of the elevation map, the position of the camera, and the field of view of the robot

In [17]:
import rospy
import math
import tf
import geometry_msgs.msg
from visualization_msgs.msg import Marker, MarkerArray
from geometry_msgs.msg import Pose, Quaternion
from std_msgs.msg import Header, Float32MultiArray, MultiArrayLayout, MultiArrayDimension
from geometry_msgs.msg import Point

br = tf.TransformBroadcaster()

In [12]:
## Put the robot model to origin
br.sendTransform((0, 0, 0),
    tf.transformations.quaternion_from_euler(0, 0, 0),
    rospy.Time(rospy.Time.now().to_sec()+0.001),
    "base",
     "map")

def get_frames_from_tf_buffer(tf_buffer):
    def extract_frame_and_parent(frameline):
        """
        The frameline is something like:
        Frame camera_init exists with parent camera_init_CORRECTED.
        """
        frameline = frameline[:-1].split()
        return frameline[1], frameline[-1]

    frame_maps = [
        extract_frame_and_parent(line)
        for line in 
            tf_buffer.all_frames_as_string().split('\n')
        if line[:5] == "Frame"
    ]
    frame_parent_maps = {f:p for f,p in frame_maps}
    # frame_parent_maps
    frame_child_maps = {}
    for f,p in frame_parent_maps.items():
        s = frame_child_maps.get(p,set())
        s.add(f)
        frame_child_maps[p] = s
    return frame_parent_maps, frame_child_maps

def child_links(base, frame_child_maps):
    que = [base]
    while(len(que)):
        base = que.pop()
        for c in frame_child_maps.get(base,[]):
            que.append(c)
            yield c, base


frame_parent_maps, frame_child_maps = get_frames_from_tf_buffer(player.tf_buffer)
for f,p in child_links("base", frame_child_maps):
    transf = player.tf_buffer.lookup_transform_core(p, f, rospy.Time(0))
    pose = msg_to_pose(transf)  # pose in fixed ref frame (odom or map)
    br.sendTransform(pose[:3],
        pose[3:],
        rospy.Time(rospy.Time.now().to_sec()+1),
        f,p)

In [23]:
# Draw the grid map
from grid_map_msgs.msg import GridMap, GridMapInfo
gp_surface_pub = rospy.Publisher("elev/dumm_grid_map", GridMap, queue_size=1);

res = 0.04
len_x = len_y = 8
gpmap = np.zeros((int(len_x/res), int(len_y/res)))

gpmapmsg = GridMap(
    info = GridMapInfo(
        header= Header(frame_id="map"),
        resolution = res,
        length_x = len_x,
        length_y = len_y,
        pose = Pose(
            position = Point(0,0,-2),
            orientation = Quaternion(x=0,y=0,z=0,w=1)
        )),
    layers = ["elevation"],
    basic_layers = ["elevation"],
    data = [
        Float32MultiArray(
            layout = MultiArrayLayout(
                dim = [
                    MultiArrayDimension(
                        label= ["column_index", "row_index"][i],
                        size = d,
                        stride = np.prod( gpmap.shape[i:])
                    )for i,d in enumerate(gpmap.shape)
                ]
            ),
            data = gpmap.reshape(-1).tolist()
            
        )
    ]
)
gp_surface_pub.publish(gpmapmsg)

In [24]:
# Draw the camera FOV
from semantic_front_end_filter.adabins.pointcloudUtils import RaycastCamera
from scipy.spatial.transform import Rotation
raycastCamera = RaycastCamera()
camera = raycastCamera.camera
camera.update_pose_from_base_pose([0,0,0, 0,0,0,1])
pose = camera.pose
cam_fx = camera.camera_matrix[0,0]
cam_fy = camera.camera_matrix[1,1]
cam_h = camera.image_height
cam_w = camera.image_width

fovx = 2*np.arctan(cam_w/(2*cam_fx))
fovy = 2*np.arctan(cam_h/(2*cam_fy))
heading = camera.pose[1][:3, 2] * 5

rot1 = Rotation.from_rotvec(fovx/2 * np.array([0, 0, 1])).as_matrix()
rot2 = Rotation.from_rotvec(-fovx/2 * np.array([0, 0, 1])).as_matrix()

head1 = rot1 @(heading-pose[0]) + pose[0]
head2 = rot2 @(heading-pose[0]) + pose[0]
axis_pub = rospy.Publisher("elev/fovaxis", MarkerArray, queue_size=1)
marr = MarkerArray()
marr.markers = [
    Marker(
        header = Header(frame_id = "map"),
        ns = "fov",
        color = type(Marker().color)(
            0,0,0,1
        ),
        scale = type(Marker().scale)(
            0.05, 0.1, 0
        ),
        id = i,
        points = [Point(*pose[0]) ]
    )
    for i in range(2)
]
marr.markers[0].type = Marker.ARROW
marr.markers[1].type = Marker.ARROW

marr.markers[0].points.append(Point(*head1))
marr.markers[1].points.append(Point(*head2))

axis_pub.publish(marr)

In [26]:
axis_pub.publish(marr)